## Uso de la fragmentación de escritura para distribuir cargas de trabajo uniformemente

Una forma de distribuir mejor las escrituras entre un espacio de claves de particiones en Amazon DynamoDB consiste en ampliar el espacio. Esto puede hacerse de diferentes maneras. Puede agregar un número aleatorio a los valores de clave de partición para distribuir los elementos entre particiones. O puede usar un número que se calcula en función de algo que esté consultando.

### Partición con sufijos aleatorios
Una estrategia para distribuir las cargas de forma más uniforme en un espacio de claves de partición consiste en añadir un número aleatorio al final de los valores de la clave de partición. De ese modo, las escrituras se distribuyen aleatoriamente por un espacio mayor.

Por ejemplo, para una clave de partición que representara la fecha de hoy, podría elegir un número aleatorio comprendido entre 1 y 200 y concatenarlo a la fecha como sufijo. De este modo, se generarían valores de clave de partición, como 2014-07-09.1, 2014-07-09.2 y así sucesivamente hasta 2014-07-09.200. Al aplicar un número aleatorio a la clave de partición, las escrituras que se producen en la tabla de cada día se distribuyen uniformemente por varias particiones. Como resultado, se mejora el paralelismo y el rendimiento general.

Sin embargo, si quisiera leer todos los elementos de un día determinado, tendría que hacer una consulta de todos los sufijos y fusionar después los resultados. Por ejemplo, primero debería emitir una solicitud Query para el valor de la clave de partición 2014-07-09.1. A continuación, emita otra Query para 2014-07-09.2 y, así sucesivamente, hasta 2014-07-09.200. Por último, la aplicación tendría que fusionar los resultados de todas estas solicitudes Query.

### Partición con sufijos calculados
Aplicar una estrategia de aleatorización puede mejorar considerablemente el rendimiento de la escritura. Sin embargo, dificulta la lectura de un elemento concreto, ya que no es posible saber qué sufijo se utilizó al escribir el elemento. Para facilitar la lectura de elementos concretos, puede usar una estrategia diferente. En lugar de utilizar un número aleatorio para distribuir los elementos entre las particiones, puede utilizar un número que se calculará en función de algo que se quiere consultar.

Consideremos el ejemplo anterior, en el que una tabla utiliza la fecha de hoy en la clave de partición. Ahora, supongamos que cada elemento tiene un atributo OrderId (ID de pedido) accesible y que lo que suele necesitar con más frecuencia es buscar elementos por el ID de pedido además de por la fecha. Antes de que la aplicación escriba el elemento en la tabla, se podría calcular un sufijo hash basado en el ID de pedido y anexarlo a la fecha de la clave de partición. El cálculo podría generar un número comprendido entre 1 y 200 que esté bastante bien distribuido, igual que el que se genera con la estrategia aleatoria.

Probablemente, bastaría con realizar un sencillo cálculo, como el producto de los valores de punto de código UTF-8 de los caracteres del ID de pedido, módulo 200, +1. El valor de la clave de partición sería la fecha junto con el resultado del cálculo.

Con esta estrategia, las escrituras se distribuyen de manera uniforme entre los valores de clave de partición y, por lo tanto, entre las particiones físicas. Es fácil realizar una operación GetItem en un elemento y una fecha determinados, ya que se puede calcular el valor de clave de partición de un valor de OrderId concreto.

Para leer todos los elementos de un día determinado, también en este caso deberá utilizar una solicitud Query para cada una de las claves 2014-07-09.N (donde N es un valor comprendido entre 1 y 200) y la aplicación tendría que fusionar todos los resultados. El beneficio es que evitaría que un único valor de clave de partición "caliente" acaparase toda la carga de trabajo.

In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
from spdynamodb import DynamoTable
import json
from decimal import Decimal
from datetime import datetime
import random

In [2]:
dt=DynamoTable()
try:
    dt.select_table('ShardingDistributionSample')
    print(dt)
except:
    dt.create_table(
        table_name='ShardingDistributionSample',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S'
    )

- Table name: ShardingDistributionSample            
- Table arn: arn:aws:dynamodb:us-east-1:237195748584:table/ShardingDistributionSample            
- Table creation: 2023-06-23 18:31:41            
- [{'AttributeName': 'PK', 'KeyType': 'HASH'}, {'AttributeName': 'SK', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'PK', 'AttributeType': 'S'}, {'AttributeName': 'SK', 'AttributeType': 'S'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


In [5]:
today = datetime.now().strftime('%Y-%m-%d')
def generate_base(id):
    id = str(id)
    total = sum(ord(char) for char in id)
    result = (total % 200) + 1
    return result

def generate_data(total_items=300):
    count_total = 0  
    pk, sk, OrderId, ProductId, Price, Category, Region = [], [], [], [], [], [], []
    for i in range(total_items):
        if count_total == total_items:
            break
        else:
            product_id = "id-" + str(random.randint(10000, 99999))
            price = random.randint(100, 9999)
            category = random.choice(["Electronics", "Books", "Home", "Kitchen", "Sports"])
            region = random.choice(["US", "EU", "ASIA"])
            order_id = "order-" + str(random.randint(10000000, 99999999))
            pk.append(today + "." + str(generate_base(order_id)))
            sk.append(order_id)
            OrderId.append(order_id)
            ProductId.append(product_id)
            Price.append(price)
            Category.append(category)
            Region.append(region)
        
    df_main = pd.DataFrame(
        {
            "PK": pk,
            "SK": sk,
            "OrderId": OrderId,
            "ProductId": ProductId,
            "Price": Price,
            "Category": Category,
            "Region": Region
        }
    )   
    return df_main

In [6]:
df = generate_data(total_items=500)
dt.batch_pandas(df)

Para saber a que PK corresponde la orden: `order-xxxxxxxx`

Ejecutamos:

In [7]:
order = 'order-28284983'
result = (sum(ord(char) for char in order) % 200) + 1
print(result)

14


In [17]:
response = dt.query(
    pk_value=today + "." + str(result),
    sk_value=order
)
print(response['Items'])

{'ProductId': 'id-89544', 'Region': 'US', 'Price': 2811, 'SK': 'order-28284983', 'PK': '2023-06-23.14', 'Category': 'Home', 'OrderId': 'order-28284983'}
